In [1]:
%pip install -r C:\Users\AubreeWang\deep-speaker\requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import sys
import random
import re
import glob
from tqdm import tqdm
import importlib
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
import umap

repos_deep_speaker_dir = os.path.dirname(os.getcwd())
sys.path.append(repos_deep_speaker_dir)
repos_dir = os.path.dirname(repos_deep_speaker_dir)
sys.path.append(repos_dir)
from deep_speaker.test import batch_cosine_similarity
# print(sys.path)
# from sak import speaker_embeddings as se
# from sak import signal_processing as sp
# from sak import vctk


In [4]:
# the directory wherey you downloaded VCTK. 
vctk_dir = r"C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92"
# the directory where you work i.e. save speaker embeddings.
work_dir = r"C:\Users\AubreeWang\deep-speaker\deep_speaker"

# you should download the deep speaker model
# and describe the path here.
checkpoint_path = r'C:\Users\AubreeWang\deep-speaker\deep_speaker\ResCNN_triplet_training_checkpoint_265.h5'

## VCTK embedding space.

In [6]:
def load_vctk_metadata():
    with open(os.path.join(vctk_dir, 'speaker-info.txt')) as f:
        lines = f.read().strip().split('\n')
    
    # remove the header.
    # ID  AGE  GENDER  ACCENTS  REGION COMMENTS
    header = ['id', 'age', 'gender', 'accent', 'region']
    lines = lines[1:]

    ## make a database of metadata.
    meta = pd.DataFrame([], columns=header)
    for i, line in enumerate(lines):
        result = re.findall(r'(\w+)\s+(\d+)\s+(M|F)\s+(\w+)\s*([\w\s]*)', line)
        result = {result[0]}
        meta = pd.concat([meta, pd.DataFrame(result, columns=header)], ignore_index=True)

    return meta

In [7]:
vctk_wavs_dir = os.path.join(vctk_dir, 'wav16')
vctk_embeddings_dir = os.path.join(work_dir, 'embeddings', 'vctk')

# check vctk meta data. 
meta = load_vctk_metadata()
meta

,id,age,gender,accent,region
0,p225,23,F,English,Southern England
1,p226,22,M,English,Surrey
2,p227,38,M,English,Cumbria
3,p228,22,F,English,Southern England
4,p229,23,F,English,Southern England
...,...,...,...,...,...
105,p363,22,M,Canadian,Toronto
106,p364,23,M,Irish,Donegal
107,p374,28,M,Australian,English
108,p376,22,M,Indian,


In [8]:
## analyse the data
meta_ = meta[meta['accent'] == 'English']
c = Counter(meta_['gender'])
c.most_common()

[('F', 18), ('M', 15)]

In [7]:
# meta_

,id,age,gender,accent,region
0,p225,23,F,English,Southern England
1,p226,22,M,English,Surrey
2,p227,38,M,English,Cumbria
3,p228,22,F,English,Southern England
4,p229,23,F,English,Southern England
5,p230,22,F,English,Stockton
6,p231,23,F,English,Southern England
7,p232,23,M,English,Southern England
8,p233,23,F,English,Staffordshire
10,p236,23,F,English,Manchester


## extract speaker embeddings.

In [9]:
## test the sample code given by the author.
import random
import os
import sys
import numpy as np

repos_deep_speaker_dir = os.path.dirname(os.getcwd())
sys.path.append(repos_deep_speaker_dir)
repos_dir = os.path.dirname(repos_deep_speaker_dir)
sys.path.append(repos_dir)

from deep_speaker.audio import read_mfcc
from deep_speaker.batcher import sample_from_mfcc
from deep_speaker.constants import SAMPLE_RATE, NUM_FRAMES
from deep_speaker.conv_models import DeepSpeakerModel
from deep_speaker.test import batch_cosine_similarity

# Reproducible results.
np.random.seed(123)
random.seed(123)

# Define the model here.
model = DeepSpeakerModel()

# Load the checkpoint. https://drive.google.com/file/d/1F9NvdrarWZNktdX9KlRYWWHDwRkip_aP.
# Also available here: https://share.weiyun.com/V2suEUVh (Chinese users).
model.m.load_weights(r"C:\Users\AubreeWang\deep-speaker\deep_speaker\ResCNN_triplet_training_checkpoint_265.h5", by_name=True)

# Sample some inputs for WAV/FLAC files for the same speaker.
# To have reproducible results every time you call this function, set the seed every time before calling it.
# np.random.seed(123)
# random.seed(123)
mfcc_001 = sample_from_mfcc(read_mfcc(r"C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92\wav48_silence_trimmed\p285\p285_288_mic2.wav", SAMPLE_RATE), NUM_FRAMES)
mfcc_002 = sample_from_mfcc(read_mfcc(r"C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92\wav48_silence_trimmed\p285\p285_288_mic1.wav", SAMPLE_RATE), NUM_FRAMES)

# Call the model to get the embeddings of shape (1, 512) for each file.
predict_001 = model.m.predict(np.expand_dims(mfcc_001, axis=0))
predict_002 = model.m.predict(np.expand_dims(mfcc_002, axis=0))
print(predict_001)

# Do it again with a different speaker.
mfcc_003 = sample_from_mfcc(read_mfcc(r"C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92\wav48_silence_trimmed\p343\p343_018_mic1.wav", SAMPLE_RATE), NUM_FRAMES)
predict_003 = model.m.predict(np.expand_dims(mfcc_003, axis=0))

# Compute the cosine similarity and check that it is higher for the same speaker.
print('SAME SPEAKER', batch_cosine_similarity(predict_001, predict_002)) # SAME SPEAKER [0.81564593]
print('DIFF SPEAKER', batch_cosine_similarity(predict_001, predict_003)) # DIFF SPEAKER [0.1419204]

1/1 [==============================] - 0s 74ms/step
[[-2.72454489e-02 -1.95157528e-02 -8.88549685e-02 -2.00686641e-02
  -3.85195836e-02 -5.85728213e-02  8.68173912e-02  2.00200342e-02
  -6.47342950e-02 -1.50347082e-02  1.06163044e-02 -5.34808785e-02
  -9.37096868e-03 -4.58109006e-02  2.90519781e-02 -8.42072535e-03
   9.95320641e-03  4.32793722e-02  7.75144086e-04 -3.93972285e-02
   2.91589391e-03  3.22580040e-02 -2.84343981e-03 -2.99638733e-02
  -1.89936869e-02 -5.73117100e-02 -7.36290142e-02  6.76643848e-02
  -2.82301474e-02  4.68667261e-02  1.90487672e-02 -4.82639261e-02
  -4.05614488e-02 -5.36496285e-04  2.83952877e-02  1.31255509e-02
  -3.40852663e-02 -4.99339290e-02  1.20050451e-02  2.75317859e-02
  -2.41500908e-03 -7.70375691e-03  1.91389620e-02  3.11033707e-02
   8.27183872e-02 -2.01727562e-02 -1.82932697e-03 -8.76876339e-02
   1.68877207e-02  1.55421980e-02 -3.46522555e-02 -8.97415876e-02
  -9.63718258e-03 -1.48889925e-02  3.68838720e-02  2.16763373e-02
   6.63513020e-02  6.799

In [10]:
## test the sample code given by the author.
import random
import os
import sys
import numpy as np

repos_deep_speaker_dir = os.path.dirname(os.getcwd())
sys.path.append(repos_deep_speaker_dir)
repos_dir = os.path.dirname(repos_deep_speaker_dir)
sys.path.append(repos_dir)

from deep_speaker.audio import read_mfcc
from deep_speaker.batcher import sample_from_mfcc
from deep_speaker.constants import SAMPLE_RATE, NUM_FRAMES
from deep_speaker.conv_models import DeepSpeakerModel
from deep_speaker.test import batch_cosine_similarity

# Reproducible results.
np.random.seed(123)
random.seed(123)

# Define the model here.
model = DeepSpeakerModel()

# Load the checkpoint. https://drive.google.com/file/d/1F9NvdrarWZNktdX9KlRYWWHDwRkip_aP.
# Also available here: https://share.weiyun.com/V2suEUVh (Chinese users).
model.m.load_weights(r"C:\Users\AubreeWang\deep-speaker\deep_speaker\ResCNN_triplet_training_checkpoint_265.h5", by_name=True)

## make a function which extract speaker embeddings from a wav file.
def extract_speaker_embedding(wav_path):
    mfcc = sample_from_mfcc(read_mfcc(f"{wav_path}", SAMPLE_RATE), NUM_FRAMES)
    speaker_embedding = model.m.predict(np.expand_dims(mfcc, axis=0))
    return speaker_embedding

In [37]:
## test function
# se=extract_speaker_embedding(r"C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92\wav48_silence_trimmed\p285\p285_288_mic1.wav")
# print(se.shape)
# se

1/1 [==============================] - 0s 84ms/step
(1, 512)


array([[-2.10564747e-03, -4.73515689e-02, -7.63562173e-02,
        -9.72307380e-03, -4.35859375e-02, -5.47640137e-02,
         1.13779269e-01,  1.86459608e-02, -2.26025023e-02,
        -7.42125418e-03,  3.03430799e-02, -3.67015600e-02,
         1.67594906e-02, -6.27308711e-02,  4.35035350e-03,
         2.88220644e-02, -1.42225646e-03,  1.97319984e-02,
         1.73367292e-03, -4.11254242e-02, -7.14668166e-03,
         5.19471169e-02,  2.74381097e-02,  8.79929308e-03,
        -1.29123262e-04, -3.97373103e-02, -5.86465299e-02,
         1.05155341e-01,  9.19337850e-04,  2.77876817e-02,
         5.62062003e-02, -1.44035518e-02, -5.99591807e-02,
        -1.84033718e-02,  4.86905724e-02,  1.30390907e-02,
        -6.03908766e-03, -3.37847322e-02, -1.39290458e-02,
        -1.61842280e-03,  2.49990690e-02, -2.03037746e-02,
        -3.61101627e-02,  4.15318608e-02,  3.50496583e-02,
        -4.97043133e-02, -7.48372031e-03, -9.20072049e-02,
         2.76544802e-02,  2.98205446e-02, -3.09270583e-0

In [40]:
from pathlib import Path

speaker_dirs = meta_["id"].tolist()
dir_path = Path(r"C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92\wav48_silence_trimmed")

filenames=[]
for speaker_dir in speaker_dirs:
    for filepath in dir_path.glob(f"{speaker_dir}/*.wav"):
        filenames.append(filepath)

print(len(filenames))
print(filenames[20000])
filenames[20000][1]




27272
C:\Users\AubreeWang\Downloads\DS_10283_3443\VCTK-Corpus-0.92\wav48_silence_trimmed\p273\p273_257_mic1.wav


TypeError: 'WindowsPath' object is not subscriptable

## visualize.

In [31]:
# use t-sne or UMAP to visualize the speaker embeddings.